**16.Calcule la probabilidad de las palabras para los textos, luego encuentre el documento que más se desvie de esas probabilidades utilizando la divergencia de Kullback-Leibler. (⭐⭐)**

In [1]:
import pandas as pd 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from scipy.stats import entropy 
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stopwordsEsp = (stopwords.words('spanish'))

In [4]:
df_textos = pd.read_csv("/content/drive/MyDrive/datosTP1/contents_text_sample.csv")

In [5]:
df_textos.head()

,id,title,text
0,50,Anacardiaceae,{{Ficha de taxón\n| name = Anacardiáceas\n| im...
1,62,Antipartícula,[[Archivo:Materia antimateria.png|thumb|200px|...
2,78,Año bisiesto,'''Año bisiesto''' es una expresión que deriva...
3,127,Avena,{{otros usos|Avena sativa|la avena común}}\n{{...
4,160,Alfabeto,{{distinguir|analfabeto|alfabetización}} \n[[A...


In [6]:
df_textos.text.isna().any()

True

In [7]:
df_textos_validos = df_textos.loc[ df_textos.text.notna() ]

In [8]:
df_textos_validos.text

0         {{Ficha de taxón\n| name = Anacardiáceas\n| im...
1         [[Archivo:Materia antimateria.png|thumb|200px|...
2         '''Año bisiesto''' es una expresión que deriva...
3         {{otros usos|Avena sativa|la avena común}}\n{{...
4         {{distinguir|analfabeto|alfabetización}} \n[[A...
                                ...                        
206602              #REDIRECCIÓN [[Nicoleta-Ancuța Bodnar]]
206603    {{Ficha de deportista\n|nombre              = ...
206604    {{Ficha de entidad subnacional\n|nombre comple...
206605     #REDIRECCIÓN [[Utilización de recursos in situ]]
206606           #REDIRECCIÓN [[IA -Aria on the Planetes-]]
Name: text, Length: 206600, dtype: object

Defino el CountVectorizer, con un patrón que busque conjunto de caracteres de más de 2 caracteres. Quito las stopwords que no aportan, y como no se especifica un límite de palabras más comunes uso 20000 que me parece suficiente para más o menos tener una idea de los textos, ya que no son demasiado largos (y no me lleva muchos minutos en el cálculo de la divergencia).

In [9]:
patron_busqueda = r"\b\w{2,}\b"

countVec = CountVectorizer( token_pattern=patron_busqueda, lowercase=True, stop_words=stopwordsEsp, max_features=20000 )

Creo una matriz con las 20000 palabras más comunes en el conjunto de textos (usando como máximo las primeras 5000000 de palabras de un texto) , siendo cada fila un vector perteneciente a un texto puntual, con el número de ocurrencias de cada palabra como contenido del vector. 

In [10]:
matriz_palabras = countVec.fit_transform( df_textos_validos.text[:5000000] )

In [11]:
matriz_palabras

<206600x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 20636081 stored elements in Compressed Sparse Row format>

In [12]:
countVec.vocabulary_

{'ficha': 8317,
 'taxón': 18047,
 'name': 12982,
 'image': 10020,
 'jpg': 10781,
 'image_caption': 10024,
 'regnum': 15881,
 'plantae': 14598,
 'divisio': 6659,
 'magnoliophyta': 11824,
 'classis': 4597,
 'magnoliopsida': 11825,
 'ordo': 13725,
 'familia': 8122,
 'robert': 16287,
 'brown': 3566,
 'br': 3471,
 'subdivision_ranks': 17639,
 'géneros': 9377,
 'subdivision': 17638,
 'ver': 19207,
 'texto': 18249,
 'biología': 3265,
 'plantas': 14599,
 'esencialmente': 7582,
 'árbol': 19932,
 'arbusto': 2335,
 'perteneciente': 14377,
 'orden': 13709,
 'constituyen': 5223,
 '77': 1183,
 'género': 9376,
 'unas': 18867,
 '700': 1145,
 'especies': 7625,
 'aceptadas': 1398,
 'casi': 4067,
 '3000': 692,
 'descritas': 6248,
 'propias': 15188,
 'países': 14195,
 'tropicales': 18699,
 'ref': 15818,
 'http': 9829,
 'www': 19748,
 'theplantlist': 18281,
 'org': 13733,
 'browse': 3568,
 'the': 18267,
 'plant': 14595,
 'list': 11469,
 '2013': 530,
 'descripción': 6244,
 'árboles': 19933,
 'arbustos': 233

In [13]:
array_frecuencias_palabras_general = np.array( np.sum( matriz_palabras, axis=0 ) ).flatten()

palabras_totales = np.sum( array_frecuencias_palabras_general )
probabilidades_palabras_general = array_frecuencias_palabras_general / palabras_totales

In [14]:
np.sum(probabilidades_palabras_general)

1.0

Como para cada texto no aparecen necesariamente las 20000 palabras obtenidas, voy a tener un array con muchos ceros, para poder calcular la divergencia de K-L, ajusto el array agregandole una probabilidad muy pequeña a cada palabra que no aparece en un texto según la muestra, así no tengo ceros.


In [15]:
def ajuste_laplace(vector):
  ajuste = 1e-12

  vector = vector + ajuste
  vector_ajustado = vector / np.sum(vector)

  return vector_ajustado



divergencias_kl = []

for i in range(206600):

  array_frecuencias_puntual = matriz_palabras[i].toarray().flatten()
  
  frecuencias_puntual_ajustado = ajuste_laplace(array_frecuencias_puntual)

  probabilidades_puntual = frecuencias_puntual_ajustado / (np.sum(frecuencias_puntual_ajustado))


  divergencia_kl = entropy( probabilidades_puntual, probabilidades_palabras_general, base=2 ) 
  divergencias_kl.append(divergencia_kl)

In [16]:
len(divergencias_kl)  

206600

In [17]:
df_textos_validos.iloc[ np.argmax(divergencias_kl), : ]

id                                    2381186
title                Portal:Benín/Exploración
text     \n<categorytree>Benín</categorytree>
Name: 49904, dtype: object